In [1]:
%config InlineBackend.figure_format ='retina'
from matplotlib.dates import date2num, num2date
from matplotlib.colors import ListedColormap
from matplotlib import dates as mdates
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
from matplotlib import ticker

import pandas as pd
import numpy as np
import os

import sys

sys.path.insert(0, '../')

from global_config import config

results_dir   = config.get_property('results_dir')
data_dir      = config.get_property('data_dir')

#%% Plot Tong's default setting
SMALL_SIZE  = 22
MEDIUM_SIZE = 22
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE, family='sans-serif', serif='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels"
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text')


In [2]:
path_to_frcst_1 = os.path.join(results_dir, "forecast", "arima")
path_to_frcst_2 = os.path.join(results_dir, "forecast", "eakf_model1")
path_to_frcst_3 = os.path.join(results_dir, "forecast", "eakf_model2")
path_to_frcst_4 = os.path.join(results_dir, "forecast", "eakf_model3")
path_to_frcst_4 = os.path.join(results_dir, "forecast", "sarima")

evals_df        = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation", "BD_scores.csv"), parse_dates=["frsct_date"])
usa_df          = pd.read_csv(os.path.join(data_dir, "processed_data_us.csv"), parse_dates=["date"])
dates_forecasts = usa_df.date[11:].values


In [3]:
def create_df_ensemble(weights_df, forecast_df_list, name_models):
    e_df = [forecast_df_list[idx]* weights_df.loc[name_models[idx]]["weigth"] for idx in range(len(forecast_df_list))]
    e_df = sum(e_df)
    return e_df


In [4]:
all_past_df               = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation",  "BD_scores_all_past.csv"), parse_dates=["frsct_date"])
all_past_df["frsct_date"] = all_past_df["forecast_date"]

eq_weights_df             = pd.read_csv( os.path.join(results_dir, "forecast", "evaluation", "BD_scores_equal_weights.csv"), parse_dates=["frsct_date"])
k_past_df                 = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation",  "BD_scores_K_past.csv"), parse_dates=["frsct_date"])
k_past_df["frsct_date"]   = k_past_df["forecast_date"]

k_past                    = ['2_past_ens', '3_past_ens', '4_past_ens', '5_past_ens', '6_past_ens']
k_past_df                 = pd.concat([k_past_df, all_past_df])

In [5]:
scores_models_df = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation", "BD_scores.csv"), parse_dates=["frsct_date"])

model_stats_df = scores_models_df.copy()
ens_stats_df   = k_past_df.copy()

eq_weights_resume_df = eq_weights_df

ens_stats_df         = ens_stats_df[ens_stats_df.method.isin(["2_past_ens", "3_past_ens", "4_past_ens", "5_past_ens", "6_past_ens", "all_past_ens"])]
ens_stats_df = ens_stats_df[ens_stats_df.eval_horizon_ensemble=="6m"]
plot_bars_df = pd.concat([model_stats_df, ens_stats_df, eq_weights_resume_df])


In [6]:
years          = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', "2021", "2022"]
pre_vac_years  = years[:6]
post_vac_years = years[6:]


In [7]:
name_cols    = {"arima": "Arima", "sarima": "Sarima", "eakf_model1": "Model 1", "eakf_model2": "Model 2", "eakf_model3": "Model 3",
                "2_past_ens": "Ens. (2 Months)", "3_past_ens": "Ens. (3 Months)", "4_past_ens": "Ens. (4 Months)",
                "5_past_ens": "Ens. (5 Months)", "6_past_ens": "Ens. (6 Months)", "all_past_ens": "Ens. (All past)", "equal_weights_ensemble": "Ens. (Eq. Weights)"}

plot_bars_df["method_plot"] = plot_bars_df["method"].apply(lambda x: name_cols[x])
plot_bars_df["period"]      = "All"
plot_bars_df["frsct_date"]  = pd.to_datetime(plot_bars_df["frsct_date"])
plot_bars_df['year']        = plot_bars_df.frsct_date.dt.year
plot_bars_df['year']        = plot_bars_df.year.apply(lambda x: str(x))


In [8]:
eval_horizons = ['1m', '2m', '3m', '4m', '5m', '6m']
models_plt = ["arima", "sarima", "eakf_model1", "eakf_model2", "eakf_model3"]


In [9]:
import scipy.stats as stats

eh = eval_horizons[0]

for eh in eval_horizons:
    plt_bar_df = plot_bars_df[plot_bars_df.method.isin(models_plt)]
    data_test  = plt_bar_df[plt_bar_df.eval_horizon==eh]

    wis_arima  = data_test[data_test.method=="arima"].wis.values
    wis_sarima = data_test[data_test.method=="sarima"].fillna(100).wis.values
    wis_model1 = data_test[data_test.method=="eakf_model1"].wis.values
    wis_model2 = data_test[data_test.method=="eakf_model2"].wis.values
    wis_model3 = data_test[data_test.method=="eakf_model3"].wis.values


    #perform the Wilcoxon-Signed Rank Test
    pval_1 = stats.wilcoxon(wis_arima, wis_model1).pvalue
    pval_2 = stats.wilcoxon(wis_arima, wis_model2).pvalue
    pval_3 = stats.wilcoxon(wis_arima, wis_model3).pvalue

    pval_4 = stats.wilcoxon(wis_sarima, wis_model1).pvalue
    pval_5 = stats.wilcoxon(wis_sarima, wis_model2).pvalue
    pval_6 = stats.wilcoxon(wis_sarima, wis_model3).pvalue


    print("Eval horizon {}. ARIMA vs Model 1 {}, ARIMA vs Model 2 {}, ARIMA vs Model 3 {}".format(eh, pval_1, pval_2, pval_3))
    print("Eval horizon {}. SARIMA vs Model 1 {}, ARIMA vs Model 2 {}, ARIMA vs Model 3 {}\n".format(eh, pval_4, pval_5, pval_6))

Eval horizon 1m. ARIMA vs Model 1 1.2900287639798705e-21, ARIMA vs Model 2 3.729620757764309e-18, ARIMA vs Model 3 1.0435802935494522e-21
Eval horizon 1m. SARIMA vs Model 1 1.0659972466585127e-15, ARIMA vs Model 2 4.525303683875214e-13, ARIMA vs Model 3 1.1635453195205961e-17

Eval horizon 2m. ARIMA vs Model 1 2.8844387350028237e-25, ARIMA vs Model 2 5.203694539137343e-24, ARIMA vs Model 3 2.4403911751829902e-26
Eval horizon 2m. SARIMA vs Model 1 7.183882677351406e-17, ARIMA vs Model 2 3.446385065856937e-17, ARIMA vs Model 3 2.85610429961773e-20

Eval horizon 3m. ARIMA vs Model 1 2.315635463841003e-28, ARIMA vs Model 2 5.323620965221437e-27, ARIMA vs Model 3 1.0367081572001511e-29
Eval horizon 3m. SARIMA vs Model 1 1.9388900261310814e-18, ARIMA vs Model 2 1.1829563752831165e-19, ARIMA vs Model 3 2.2421255349506346e-22

Eval horizon 4m. ARIMA vs Model 1 5.907229992530113e-29, ARIMA vs Model 2 5.06199207481235e-29, ARIMA vs Model 3 5.9273150836936e-31
Eval horizon 4m. SARIMA vs Model 1 5

In [10]:
for eh in eval_horizons:
    plt_bar_df = plot_bars_df[plot_bars_df.method.isin(models_plt)]
    data_test  = plt_bar_df[plt_bar_df.eval_horizon==eh]

    wis_arima  = data_test[data_test.method=="arima"].wis.values
    wis_sarima = data_test[data_test.method=="sarima"].fillna(100).wis.values
    wis_model1 = data_test[data_test.method=="eakf_model1"].wis.values
    wis_model2 = data_test[data_test.method=="eakf_model2"].wis.values
    wis_model3 = data_test[data_test.method=="eakf_model3"].wis.values


    pval_1 = stats.wilcoxon(wis_arima, wis_model1).pvalue
    pval_2 = stats.wilcoxon(wis_arima, wis_model2).pvalue
    pval_3 = stats.wilcoxon(wis_arima, wis_model3).pvalue

    print("{} &  {:.2e} & {:.2e} & {:.2e} \\\\".format(eh, pval_1, pval_2, pval_3))

1m &  1.29e-21 & 3.73e-18 & 1.04e-21 \\
2m &  2.88e-25 & 5.20e-24 & 2.44e-26 \\
3m &  2.32e-28 & 5.32e-27 & 1.04e-29 \\
4m &  5.91e-29 & 5.06e-29 & 5.93e-31 \\
5m &  3.89e-29 & 3.09e-30 & 1.37e-31 \\
6m &  4.98e-29 & 2.05e-30 & 2.23e-31 \\


In [11]:
for eh in eval_horizons:
    plt_bar_df = plot_bars_df[plot_bars_df.method.isin(models_plt)]
    data_test  = plt_bar_df[plt_bar_df.eval_horizon==eh]

    wis_arima  = data_test[data_test.method=="arima"].wis.values
    wis_sarima = data_test[data_test.method=="sarima"].fillna(100).wis.values
    wis_model1 = data_test[data_test.method=="eakf_model1"].wis.values
    wis_model2 = data_test[data_test.method=="eakf_model2"].wis.values
    wis_model3 = data_test[data_test.method=="eakf_model3"].wis.values


    pval_4 = stats.wilcoxon(wis_sarima, wis_model1).pvalue
    pval_5 = stats.wilcoxon(wis_sarima, wis_model2).pvalue
    pval_6 = stats.wilcoxon(wis_sarima, wis_model3).pvalue

    print("{} &  {:.2e} & {:.2e} & {:.2e} \\\\ \n".format(eh, pval_4, pval_5, pval_6))

1m &  1.07e-15 & 4.53e-13 & 1.16e-17 \\ 

2m &  7.18e-17 & 3.45e-17 & 2.86e-20 \\ 

3m &  1.94e-18 & 1.18e-19 & 2.24e-22 \\ 

4m &  5.76e-20 & 9.77e-22 & 6.91e-25 \\ 

5m &  1.51e-21 & 1.56e-23 & 2.14e-26 \\ 

6m &  1.32e-22 & 3.56e-24 & 2.11e-28 \\ 



In [12]:

bars_all_period_df = []

name_cols    = {"arima": "Arima", "sarima": "Sarima", "eakf_model1": "Model 1", "eakf_model2": "Model 2", "eakf_model3": "Model 3",
                "2_past_ens": "Ens. (2 Months)", "3_past_ens": "Ens. (3 Months)", "4_past_ens": "Ens. (4 Months)",
                "5_past_ens": "Ens. (5 Months)", "6_past_ens": "Ens. (6 Months)", "all_past_ens": "Ens. (All past)", "equal_weights_ensemble": "Ens. (Eq. Weights)"}

plot_bars_df["method_plot"] = plot_bars_df["method"].apply(lambda x: name_cols[x])
plot_bars_df["period"]      = "All"
plot_bars_df["frsct_date"]  = pd.to_datetime(plot_bars_df["frsct_date"])
plot_bars_df['year']        = plot_bars_df.frsct_date.dt.year
plot_bars_df['year']        = plot_bars_df.year.apply(lambda x: str(x))

bars_all_period_df.append(plot_bars_df)

models_plt = ["eakf_model3", "2_past_ens", "all_past_ens", "equal_weights_ensemble"]

plt_bar_df = plot_bars_df[plot_bars_df.method.isin(models_plt)][["wis", "eval_horizon", "frsct_date", "method", "period"]]

eh        = eval_horizons[0]

models = [("eakf_model3", "2_past_ens"), ("eakf_model3", "all_past_ens"), ("2_past_ens", "all_past_ens"),
            ("eakf_model3", "equal_weights_ensemble"), ("2_past_ens", "equal_weights_ensemble"), ("all_past_ens", "equal_weights_ensemble")]

pvals_df = pd.DataFrame(columns=["model1", "model2", "pval", "eh"])
for eh in eval_horizons:
    data_test = plt_bar_df[plt_bar_df.eval_horizon==eh]
    for model1, model2 in models:
            dates1 = data_test[data_test.method==model1].frsct_date
            dates2 = data_test[data_test.method==model2].frsct_date
            dates  = list(set(dates1).intersection(dates2))

            data_use   = data_test[data_test.frsct_date.isin(dates)]
            wis_model1 = data_use[data_use.method==model1].fillna(100).wis.values
            wis_model2 = data_use[data_use.method==model2].fillna(100).wis.values

            pval       = stats.wilcoxon(wis_model1, wis_model2).pvalue

            pvals_df= pvals_df.append({"model1": model1, "model2": model2, "pval": pval, "eh": eh}, ignore_index=True)

pvals_df["column_latex"] = pvals_df.apply(lambda x: "{} vs {}".format(name_cols[x.model1], name_cols[x.model2]), axis=1)
pvals_df

,model1,model2,pval,eh,column_latex
0,eakf_model3,2_past_ens,4.072814e-01,1m,Model 3 vs Ens. (2 Months)
1,eakf_model3,all_past_ens,8.009813e-01,1m,Model 3 vs Ens. (All past)
2,2_past_ens,all_past_ens,7.541345e-01,1m,Ens. (2 Months) vs Ens. (All past)
3,eakf_model3,equal_weights_ensemble,6.310703e-13,1m,Model 3 vs Ens. (Eq. Weights)
4,2_past_ens,equal_weights_ensemble,1.068200e-08,1m,Ens. (2 Months) vs Ens. (Eq. Weights)
5,all_past_ens,equal_weights_ensemble,3.368874e-12,1m,Ens. (All past) vs Ens. (Eq. Weights)
6,eakf_model3,2_past_ens,8.221228e-02,2m,Model 3 vs Ens. (2 Months)
7,eakf_model3,all_past_ens,2.427269e-01,2m,Model 3 vs Ens. (All past)
8,2_past_ens,all_past_ens,8.577363e-02,2m,Ens. (2 Months) vs Ens. (All past)
9,eakf_model3,equal_weights_ensemble,4.207459e-18,2m,Model 3 vs Ens. (Eq. Weights)


In [13]:
for idx_row, row in pd.pivot_table(pvals_df, index="column_latex", columns=["eh"], values="pval").iterrows():
    print("{} & {:.2e} & {:.2e} & {:.2e} & {:.2e} & {:.2e} & {:.2e} ".format(idx_row, row["1m"], row["2m"], row["3m"], row["4m"], row["5m"], row["6m"]))

Ens. (2 Months) vs Ens. (All past) & 7.54e-01 & 8.58e-02 & 6.65e-02 & 2.32e-02 & 9.06e-04 & 1.58e-03 
Ens. (2 Months) vs Ens. (Eq. Weights) & 1.07e-08 & 4.87e-11 & 7.93e-13 & 1.31e-16 & 8.09e-17 & 1.33e-17 
Ens. (All past) vs Ens. (Eq. Weights) & 3.37e-12 & 3.21e-17 & 1.82e-20 & 4.89e-23 & 1.70e-24 & 1.39e-25 
Model 3 vs Ens. (2 Months) & 4.07e-01 & 8.22e-02 & 6.09e-02 & 2.13e-02 & 1.36e-03 & 7.28e-03 
Model 3 vs Ens. (All past) & 8.01e-01 & 2.43e-01 & 2.02e-01 & 1.14e-01 & 1.07e-01 & 5.60e-01 
Model 3 vs Ens. (Eq. Weights) & 6.31e-13 & 4.21e-18 & 4.68e-21 & 4.46e-24 & 2.13e-25 & 9.08e-26 


In [14]:
print("{:.2e}".format(5/100))

5.00e-02
